In [1]:
import collections
import re
from d2l import torch as d2l

读取数据集

In [2]:
d2l.DATA_HUB['time_machine'] = (
    d2l.DATA_URL + 'timemachine.txt',
    '090b5e7e70c295757f55df93cb0a180b9691891a'
)

In [3]:
def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [
        re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines
    ]

In [4]:
lines = read_time_machine()
print(f"# 文本总行数：{len(lines)}")
print(lines[0])
print(lines[10])

# 文本总行数：3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


词元化

In [5]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print("错误：未知词元类型" + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


词表

In [6]:
class Vocab:
    # 文本词表
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []

        # 按出现频率排序
        counter = count_corpus(tokens)
        '''
        `counter.items()` 返回这个字典中的所有键值对，以 (key, value) 的形式构成一个可迭代的视图
        
        `sorted` 函数有一个可选参数 `key`，用于指定排序的依据。
        在这里，`key=lambda x: x[1]` 表示我们要根据每个 (key, value) 对中的 `value`（即单词的出现频率）进行排序。
        `lambda x: x[1]` 是一个匿名函数，输入是字典中的一个 (key, value) 对，输出是这个对中的 `value` 部分。

        `sorted` 函数还有一个可选参数 `reverse`，用于指定排序的顺序。
        `reverse=True` 表示我们要进行降序排序，即频率高的单词排在前面。

        '''
        self._token_freqs = sorted(
            counter.items(),
            key=lambda x: x[1],
            reverse=True
        )

        '''
        初始化未知词元索引
        
        self.idx_to_token = ['<unk>'] + reserved_tokens:初始化了一个列表idx_to_token，
        它将用于存储词汇表中的词元（tokens）及其对应的索引。
        列表的第一个元素是'<unk>'，这是一个特殊的词元，用于代表那些在词汇表中未被包含的词元（即未知词元）。
        reserved_tokens是一个参数列表，包含了一些需要保留的词元，这些词元会被添加到'<unk>'之后。

        未知词元的索引为0

        '''
        # 
        self.idx_to_token = ['<unk>'] + reserved_tokens
        '''
        创建词元到索引的映射
        
        self.token_to_idx是一个字典推导式，用于创建一个从词元到索引的映射。
        enumerate函数用于获取idx_to_token列表中每个元素的索引（idx）和值（token），
        然后字典推导式将每个词元映射到它的索引。
        '''
        self.token_to_idx = {
            token: idx
            for idx, token in enumerate(self.idx_to_token)
        }
        '''
        填充词汇表

        遍历_token_freqs列表，这个列表包含了按频率排序的词元及其出现频率。循环体内部包含两个主要的操作：
        
        if freq < min_freq: 
            break：
        如果当前词元的出现频率小于min_freq参数指定的最小频率，则跳出循环。
        这意味着只有出现频率大于或等于min_freq的词元才会被加入词汇表。

        if token not in self.token_to_idx:如果当前词元不在token_to_idx字典中，
        即它还没有被加入词汇表，那么执行以下操作：
        
            self.idx_to_token.append(token)：将词元添加到idx_to_token列表的末尾。
            
            self.token_to_idx[token] = len(self.idx_to_token) - 1：
            更新token_to_idx字典，将新词元映射到它在idx_to_token列表中的索引。
        '''
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    
    def __len__(self):
        return len(self.idx_to_token)
        
    def __getitem__(self, tokens):
        # 如果tokens不是一个列表或元组，说明用户想要获取单个词元的索引
        if not isinstance(tokens, (list, tuple)):
            # 使用字典的get方法尝试获取tokens的索引，如果tokens不在字典中，则返回未知词元索引self.unk
            return self.token_to_idx.get(tokens, self.unk)
        # 如果tokens是一个列表或元组，说明用户想要获取多个词元的索引列表
        # 对tokens中的每个元素调用__getitem__方法，并返回一个包含所有索引的列表
        return [self.__getitem__(token) for token in tokens]
    def to_tokens(self, indices):
        # 检查indices是否不是一个列表或元组
        if not isinstance(indices, (list, tuple)):
            # 如果indices不是一个列表或元组，这意味着它是一个单个的索引。
            # 因此，可以直接使用这个索引来从idx_to_token列表中获取对应的词元，并返回这个词元。
            return self.idx_to_token[indices]

        '''
        如果indices是一个列表或元组，这意味着它包含了多个索引。
        这是一个列表推导式，用于遍历indices中的每个索引index，
        使用每个索引从idx_to_token列表中获取对应的词元，并构建一个新的列表，
        包含所有找到的词元。最后，返回这个包含词元的列表。

        '''
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): #未知元素的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [7]:
def count_corpus(tokens):
    # 统计词元的频率
    if len(tokens) == 0 or isinstance(tokens[0], list):
        '''
        将词元列表展平成一个列表
        
        [token for line in tokens for token in line]
        
            for line in tokens：这部分是外部循环，它遍历tokens中的每一个元素。
                在这里，我们假设tokens是一个列表，其元素也是列表（即二维列表）。所以，line会依次取tokens中的每一个子列表。
            for token in line：这部分是内部循环，它遍历当前line（即tokens中的一个子列表）中的每一个元素。
                在这里，我们假设line是一个列表，其元素是词元（tokens）。所以，token会依次取line中的每一个词元。
        
        
        '''
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)
    

额外例子

In [8]:
vocab = Vocab(tokens)
print(
    list(
        vocab.token_to_idx.items()
    )[:10]
)

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [9]:
for i in [0, 10]:
    print('文本：',tokens[i])
    print('索引：',vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引： [1, 19, 50, 40, 2183, 2184, 400]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引： [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


整合

In [10]:
def load_corpus_time_machine(max_tokens=-1):
    # 返回时光机器数据集的词元索引列表和词表
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机数据集中的每个文本行不一定是句子或者一个段落
    # 所以将所有文本行展开到一个列表中
    corpus = [
        vocab[token] for line in tokens for token in line
    ]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

In [11]:
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)